# Pytest

Testy již určitě znáte z domácích úloh. Pravidlem je začínat název testovací funkce prefixem **test_**. Díky tomu to rozpozná i Vaše IDE.  Je to svým způsobem dokumentace vaší budoucí implementace - z testu vidíme jaká funkce co vrací. Zároveň je dobré dodržovat následující pravidlo - funkce by měla řešit jednoduchou úlohu == lze jednoduše otestovat. Někteří programátoři prosazují metodu **test driven development (TDD)**. To je metoda, kde se nejdříve napíší testy (které samozřejmě budou failovat) a postupně píšeme kód tak, aby naše testy procházely.

Dobrý materiál:
https://naucse.python.cz/lessons/intro/testing/ (nezapíráme inspiraci)


In [ ]:
# an ancient pytest (3.4.6) is preinstalled by default
!pytest --version
# lets upgrade
!pip install --upgrade pytest
!pytest --version

## Základní testy

Testy se píší do souborů pojmenovaných **test_{cokoli}.py** a umisťují se buď vedle adresáře s testovaným modulem, nebo přímo do něj. Pokud volíte druhou možnost, pamatujte, že nebudou fungovat relativní importy, protože testy jsou pouštěny jako samostatné moduly. Lepší je tak první možnost (mít testy vedle modulu; tedy např. v adresáři **tests**).

In [ ]:
%%writefile f.py 
def my_difficult_func(a, b, operator):
  # eval takes python code as a string and evaluate it
  return int(eval(f'a{operator}b'))

In [ ]:
%%writefile test_f1.py 
from f import my_difficult_func

def test_my_difficult_func_simple():
  # the simplest test
  # expectation
  expected = 3
  # real result
  result = my_difficult_func(1, 2, '+')
  # comparison
  assert expected == result

Výše uvedený kód ukazuje základní strukuru jak by měl každý test vypadat. Tedy vždy si určíme co bychom chtěli aby naše funkce vracela, poté zavoláme naší implementaci funkce a vždy končíme porovnáním zdali jsme získali to, co jsme získat chtěli. Na to nám slouží klíčové slovo **assert**. To je v podstatě podmínka - pokud výsledky nesedí dostaneme exception a rovnou vidíme i jak se výsledky rozchází. 

Tím, že jsme si funkci pojmenovali **test_{cokoli}()**, tak pytest sám pozná, že jde o definici testu.

In [ ]:
!ls

In [ ]:
# Alternatively we can run it as python module: python -m pytest
!pytest test_f1.py
#!ls -la
!python -m pytest

Většinou se píší testy tzv. pozitivní (testujeme to co chceme, aby naše funkce dělala), negativní (testujeme např. nevalidní vstupy), hraniční (do funkce posíláme sporné validní vstupy - znáte z Progtestu™).

## Parametrizované testy

První ukázka testu má jednu slabinu, testujeme jen jeden vstup `(1, 2, '+')`. Pro více vstupů bychom opakovali kód. Tady nastupuje síla pytest, kde můžeme poslat na jeden test více vstupů. Syntaxe je jednoduchá, první parametr dekorátoru je **název parametru v testovací funkci** (string) a pokud je jich více, tak je to string s jejich jmény oddělenými čárkou. Druhým parametrem dekorátoru je **iterable (typicky list), které obsahuje hodnoty**, jež se mají za daný parametr dosazovat. Pokud je parametrů více, jsou hodnoty v iterable jako tuple (musí sedět počet parametrů a hodnot v tuple).


In [ ]:
%%writefile test_f2.py 
from f import my_difficult_func
import pytest

# improvement - we can test multiple inputs
# 1st decorator arg ~ argument name
# 2nd decorator arg ~ iterable
@pytest.mark.parametrize('a', ['red', 'green', 'blue'])
def test_dummy_builtin(a):
    assert len(a) > 0 and len(a) < 6
    # we can have more asserts in one test, all must be satisfied to pass
    assert 666 > 42

@pytest.mark.parametrize(
    'a, b, operator, expected',
    [
        (3, 5, "+", 8),
        (6, 9, "*", 54),
        (1, 6, "-", -5),
        (10, 5, "/", 2),
        (15, 9, "%", 6),
    ])
def test_func_multi(a, b, operator, expected):
    assert expected == my_difficult_func(a, b, operator)

In [ ]:
# we can explicitly run only some test, otherwise pytest recursively look all test_* files in working dir
!pytest

# -v = verbose
!pytest -v test_f2.py

**TIP:** Abychom se vyhnuli psaní testů na testy, není od věci, dočasně testovaný kód "rozbít" a ověřit, zda rozbití test zachytil.

## Fixtures
Je postup, který využijeme k tomu, když před samotným spuštěním testu potřebujeme něco předpřipravit - například získat připojení do databáze, inicializace objektu, či načtení dat ze souboru.

Fixture je objekt, který vrací námi definovaná funkce anotovaná pomoci `@pytest.fixture`. Do testu ji předáme parametrem a podle názvu tohoto parametru se hledá definice dané fixture. Pokud potřebujeme například spojení k DB slušně ukončit, použijme ve funkci definující fixture `yield` místo `return` a po yieldu máme možnost po sobě zamést (eg. `db.close()`).



In [ ]:
%%writefile test_f3.py 
from f import my_difficult_func as func
import pytest

@pytest.fixture
def difficult_prepare():
    # simulation of difficult preparation. It's usually used ie. for loading files, reading images,...
    return 54

def test_with_fixture(difficult_prepare):
  # the same name of parameter as our fixture
  print("This is by default captured by pytest and hidden...")
  assert difficult_prepare == func(6, 9, "*")

Výše uvedený kód je spíš jen ukázka (ne příliš vhodného) využití **pytest fixture**. Znáte ho třeba z 2. domácího úkolu, kde se fixture používal na read_image. V samotném testu jsme pak již měli výsledný obrázek jako numpy array a mohli ho porovnávat s naším výsledkem.

In [ ]:
!pytest -v test_f3.py

# -s = show content from stderr and stdout
!pytest -v -s test_f3.py

I samotné fixtury jdou parametrizovat, ale argumenty do dekorátoru se musí předat pojmenovaným parametrem **params**. Konkrétní hodnota jde pak získat přes `request.param`.

In [ ]:
%%writefile test_f4.py

import pytest
import os

# parametrized fixture must have exact parameter name == 'request'
@pytest.fixture(params=['shark', 'octopus'])
def measure_length(request):
  # heavy load... animal seeking, hunting, measuring...
  return len(request.param)

def test_animal_length(measure_length):
  print(f"function test_animal_length says: value of measure_length returned by fixture = {measure_length}")
  try:
    print(os.environ['MY_SECRET_TOKEN'])
  except KeyError:
    print('BTW: No MY_SECRET_TOKEN defined in environment variables!')
  # animals cannot be zero-length
  assert measure_length > 0

**TIP:** Někdy je potřeba mít v testech citlivé údaje (heslo k DB, token k API), pak lze využít proměnné prostředí (prozkoumejte os.environ)

In [ ]:
!pytest -v -s test_f4.py
# in colab, we cannot use this because it is runned in subshell
!export MY_SECRET_TOKEN="..."
# so %magic is handy
%env MY_SECRET_TOKEN="In BI-PYT we check HW code plagiarity."
!pytest -s test_f4.py

## Mocking, test double

Někdy se může hodit, aby testy neměly vedlejší účinky (př.: test mazání v DB - to byste pak museli pokaždé data obnovovat). Pak se pracuje se zástupnými objekty k těm testovaným. Uděláte si třídu se stejným rozhraním jako má ta testovaná a hurá testovat, jak s ní zbytek projektu interaguje. Jen je to hromada psaní kódu.

Existuje knihovna [flexmock](https://flexmock.readthedocs.io/), která nám trochu usnadní psaní těch zástupných objektů:

In [ ]:
!pip install flexmock

In [ ]:
%%writefile test_f5.py

import pytest
from flexmock import flexmock
import builtins
import random
from io import StringIO

#-----------------------------------------
# when you are paid by kLOC...
class DummyDatabaseItem:
  id = -1
  def __init__(self, id): self.id=id
  def delete(self): pass
  def update(self): pass
  def rnd(self): return random.randint(1, 6)

@pytest.fixture
def items1():
  return [DummyDatabaseItem(x) for x in range(5)]
#-----------------------------------------

#-----------------------------------------
# more comfortable way
@pytest.fixture
def items2():
  return [flexmock(id=x, delete=lambda: None, update=lambda: None) for x in range(5)]
#-----------------------------------------

def custom_deleting():
  print('custom_deleting() called')
  return 666

def test_deleting(items2):
  for i in items2:
    assert i.delete() is None

def test_modifying_existing_objects():
  ddi = DummyDatabaseItem(1)
  assert ddi.delete() == None

  # the 1st argument can be object (instance), class, module
  flexmock(ddi, delete=42)
  assert ddi.delete() == 42

  # "real" implementation with mocked function "delete"
  flexmock(ddi).should_receive('delete').replace_with(lambda: custom_deleting())
  assert ddi.delete() == 666

# it is possible to override builtin methods
def test_override_builtins(): 
  flexmock(builtins, open=StringIO('fake content'))
  with open('/root/.ssh/id_rsa') as f:
    assert f.readlines() == ['fake content']

def test_random_can_be_seeded():
  ddi = DummyDatabaseItem(1)
  assert ddi.rnd() < 4  # change to < 10 if you want to pass everytime

def test_random_can_be_seeded_1():
  ddi = DummyDatabaseItem(1)
  random.seed(42)
  assert ddi.rnd() == 6

In [ ]:
!pytest -v -s test_f5.py

Pro omezení dosahu změn (např.: modifikace Path) na běh jednoho testu lze použít fixture **monkeypatch** ([dokumentace](https://docs.pytest.org/en/latest/monkeypatch.html)) zabudovanou přímo v pytestu. 

In [ ]:
# example of selecting only some tests (prefix matching)
!pytest -v -s test_f5.py -k test_random_can_be_seeded

**TIP:** S mockováním to nepřehánějte, abyste časem netestovali jen vaše fake objekty, funkce, moduly. Často jde upravit původní kód, aby byla jeho funkcionalita lépe testovatelná.